# GUI hbond

### Initialization

In [46]:
import os

import subprocess
import platform
import pandas as pd

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [48]:
platform.system()

'Linux'

In [49]:
os.getcwd()

'/home/juk/Python_WS21-22/hbond/alex_git/HBonds_Project/hbonds_search_pymol/linux'

### Functions

In [33]:
from pymol import cmd, stored


def form(pdbstr, x):
    '''
    Part of lambda function to format dataframe to Pymol compatible form:
    from "A:183:LEU:O" to "/2akr/A/A/LEU`183/O"
    :param pdbstr: pdb code of handled structure
    :param x: entry within dataframe
    '''
    temp = x.split(':')
    x = f'/{pdbstr}/{temp[0]}/{temp[0]}/{temp[2]}/{temp[3]}'
    return x

def pymol_display(df):
    '''
    '''
    zilis = list(zip(df['ACC'].tolist(), df['DONO'].tolist()))
    for i in zilis:
        cmd.distance( i[0] , i[1])
    
def hbsearch(pdbstr:str) -> pd.DataFrame():
    '''
    Executing hb_search with set parameters and extract HBOND-entries from output
    :return df_hbond: Dataframe with all HBOND entries from hb_search output 
    '''
    
    HEAD_LST = ['IDENT','ACC','sep1','DONO',':','x','y','z','sep2','a','b']
    
    # Setting environment variable
    os.environ['PSE_FILE'] = 'period-table-info.txt'
    
    # Executing hb_search
    hbs = subprocess.run(f"./hb-search -hb hb-define.txt {pdbstr}.pdb", 
                         stdout=subprocess.PIPE, shell=True, check=True, text=True)
    
    # Decode and format hb_search output
    hbs_hbb = [i for i in hbs.stdout.split('\n') if i[0:5] == "HBOND"]
    hbs_splt = [i.split() for i in hbs_hbb]

    # Return dataframe
    df_hbond = pd.DataFrame(hbs_splt, columns = HEAD_LST)
    
    df_hbond = df_hbond[['ACC', 'DONO']]
    df_hbond['ACC'] = df_hbond['ACC'].map(lambda x: form(pdbstr,x) )
    df_hbond['DONO'] = df_hbond['DONO'].map(lambda x: form(pdbstr,x) )
    
    pymol_display(df_hbond)
    
    return df_hbond

cmd.extend('hbsearch', hbsearch)

### Main body

In [34]:
if __name__ == '__main__':

    df_hbond = hbsearch('2akr')


In [35]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
df_hbond

,ACC,DONO
0,/2akr/A/A/ASN/OD1,/2akr/A/A/HOH/O
1,/2akr/A/A/TYR/N,/2akr/A/A/MET/O
2,/2akr/A/A/TYR/OH,/2akr/A/A/LEU/O
3,/2akr/A/A/THR/N,/2akr/A/A/HOH/O
4,/2akr/A/A/ARG/NE,/2akr/A/A/HOH/O
...,...,...
3554,/2akr/D/D/HOH/O,/2akr/D/D/HOH/O
3555,/2akr/D/D/HOH/O,/2akr/D/D/HOH/O
3556,/2akr/D/D/HOH/O,/2akr/D/D/ASN/O
3557,/2akr/D/D/HOH/O,/2akr/D/D/HOH/O
